In [1]:
from pathlib import Path
import os
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# from IPython.display import display

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras import layers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_path = 'D:/Machine Learning/train'
test_path = 'D:/Machine Learning/test'


In [4]:
file_count = sum(len(files) for _, _, files in os.walk(train_path))
file_count2 = sum(len(files) for _, _, files in os.walk(test_path))

In [5]:

print(file_count,file_count2)
batch_size = 32

img_height = 500
img_width = 500
# classifier = Sequential()

# classifier.add(Convolution2D(batch_size,3,3,input_shape=(150,150,3), activation='relu'))

# classifier.add(Dropout(.1))

# classifier.add(MaxPooling2D(pool_size=(2,2)))

# classifier.add(Flatten())

93798 94100


In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_path,
  validation_split=.2,
  subset="training",
   color_mode= 'rgb',
     seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_path,
  validation_split=.2,
  color_mode= 'rgb',
  seed=123,
  subset="validation",
  batch_size=batch_size)


Found 93798 files belonging to 6062 classes.
Using 75039 files for training.
Found 93798 files belonging to 6062 classes.
Using 18759 files for validation.


In [7]:
# classifier.add(Dense(1024,activation='relu'))
# classifier.add(Dense(512,activation='relu'))
# classifier.add(Dense(6062,activation='softmax'))

# classifier.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
class_names = train_ds.class_names

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")


ResourceExhaustedError: Failed to allocate memory for the batch of component 0

In [ ]:
num_classes = 6062

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)
# test_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#         train_path,
#         target_size=(150, 150),
#         batch_size=batch_size,
#         class_mode='categorical')
# validation_generator = test_datagen.flow_from_directory(
#         test_path,
#         target_size=(150, 150),
#         batch_size=batch_size,
#         class_mode='categorical')
# try:
#         with tf.device("/device:GPU:0"):

#                 classifier.fit_generator(
#                         train_generator,
#                         steps_per_epoch=file_count/batch_size,
#                         epochs=5,
#                         validation_data=validation_generator,
#                         validation_steps=file_count2/batch_size)
# except Exception as e:
#     print(e)

